In [1]:
% load_ext autoreload
% autoreload 2

In [9]:
from models import pretrained

import pytorch_lightning as pl
from data.sample_data.cifar import cifar10, cifar10_1
from torch.utils.data import DataLoader
from utils.inference import LabelCollector
from utils.modules import PQModule
from utils.loader import PQLoader
from data import sample_data
from data.core import split_dataset
import torch
import pandas as pd

In [6]:
# load base model
model = pretrained.resnet18_trained_on_cifar10()

In [7]:
# evaluate base model on CIFAR 10 and 10.1
tr = pl.Trainer(gpus=1)
dl = DataLoader(cifar10(normalize=True, resize=224, split='val'), batch_size=256, num_workers=16)
dl_1 = DataLoader(cifar10_1(normalize=True, resize=224), batch_size=256, num_workers=16)

module = LabelCollector(model=model)
tr.validate(module, dl, verbose=False)
print(f'CIFAR 10 Accuracy of Base Model: {100 * module.compute_accuracy():.2f}%')

module = LabelCollector(model=model)
tr.validate(module, dl_1, verbose=False)
print(f'CIFAR 10.1 Accuracy of Base Model: {100 * module.compute_accuracy():.2f}%')

/voyager/projects/tomginsberg/.conda/envs/shiftdynamics/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/voyager/projects/tomginsberg/.conda/envs/shiftdynamics/lib/python3.10/site-packages/lightning_lite/plugins/environments/slurm.py:167: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /voyager/projects/tomginsberg/.conda/envs/shiftdynam ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/voyager/projects/tomginsberg/.conda/envs/shiftdynamics/lib/python3.10/site-packages/lightning_

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


CIFAR 10 Accuracy of Base Model: 91.18%


Validation: 0it [00:00, ?it/s]

CIFAR 10.1 Accuracy of Base Model: 82.45%


In [3]:
seed = 0
N = 50

In [5]:
load_model = lambda: pretrained.resnet18_trained_on_cifar10()
p_train, p_val, p_test_all = sample_data.cifar10(split='all')
q_all = sample_data.cifar10_1()

test_sets = {'p': p_test_all, 'q': q_all}
base_model = load_model()

# hyperparams ---------------------------------------------
max_epochs_per_model = 1
optimizer = lambda params: torch.optim.Adam(params, lr=1e-3)
ensemble_size = 10
batch_size = 512
# ---------------------------------------------------------
gpus = [0]
num_workers = 16
# ---------------------------------------------------------

In [6]:
q, _ = split_dataset(test_sets['q'], N, seed)
p, _ = split_dataset(test_sets['p'], N, seed)

In [7]:
pq_loader = PQLoader(p_train=p_train,
                     p=p,
                     q=q,
                     batch_size=batch_size,
                     num_workers=num_workers,
                     )

In [8]:
alpha = 1 / (len(pq_loader.train_dataloader()) * N + 1)

In [ ]:
import logging
logging.getLogger("lightning").setLevel(logging.ERROR)

In [ ]:
results = []
detector = PQModule(load_model(), alpha=alpha)

pl.Trainer(gpus=gpus, logger=False, max_epochs=1).test(detector, pq_loader.test_dataloader(), verbose=False)
results.append(detector.test_struct.to_dict())

for i in range(20):
    trainer = pl.Trainer(
        gpus=gpus,
        max_epochs=max_epochs_per_model,
        logger=False,
        num_sanity_val_steps=0,
        limit_val_batches=0,
        enable_model_summary=False
    )
    trainer.fit(detector, pq_loader)

    pl.Trainer(gpus=gpus, logger=False, max_epochs=1).test(detector, pq_loader.test_dataloader(), verbose=False)
    results.append(detector.test_struct.to_dict())

/voyager/projects/tomginsberg/.conda/envs/shiftdynamics/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  rank_zero_deprecation(
/voyager/projects/tomginsberg/.conda/envs/shiftdynamics/lib/python3.10/site-packages/lightning_lite/plugins/environments/slurm.py:167: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /voyager/projects/tomginsberg/.conda/envs/shiftdynam ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/voyager/projects/tomginsberg/.conda/envs/shiftdynamics/lib/python3.10/site-packages/lightn

Testing: 0it [00:00, ?it/s]

/voyager/projects/tomginsberg/.conda/envs/shiftdynamics/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/voyager/projects/tomginsberg/.conda/envs/shiftdynamics/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
/voyager/projects/tomginsberg/.conda/envs/shiftdynamics/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory /voyager/projects/tomginsberg/shiftdynamics/checkpoints exists and

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


...

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: 0it [00:00, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Training: 0it [00:00, ?it/s]